# Demo on active testing from paper
Active Testing: Sample–Efficient Model Evaluation

Active Surrogate Estimators: An Active Learning Approach to Label–Efficient Model Evaluation

In [1]:
import os, sys
import torch, json
import numpy as np

from main import build_model_main
from util.slconfig import SLConfig
from datasets import build_dataset
from util.visualizer import COCOVisualizer
from util import box_ops
import pickle

In [2]:
result_path = "./results/deep_ensemble/"
consider_model_results = {"DINO_0011_4scale.pkl", "DINO_0011_5scale.pkl", "DINO_0023_4scale.pkl", "DINO_0022_5scale.pkl", "DINO_0033_4scale.pkl", "DINO_0031_5scale.pkl"}
main_model_index = 5

## Load model results

In [3]:
deep_ensemble_results = []
for path in consider_model_results:
    temp_path = result_path + path
    with open(temp_path, "rb") as outfile:
         deep_ensemble_results.append(pickle.load(outfile))

## Load data set

In [4]:
scale_config = "5scale"
model_config_path = "config/DINO/DINO_" + scale_config + ".py" # change the path of the model config file
args = SLConfig.fromfile(model_config_path) 
args.device = 'cuda' 
args.dataset_file = 'coco'
args.coco_path = "../coco/" # the path of coco
args.fix_size = False
dataset_val = build_dataset(image_set='val', args=args)

data_aug_params: {
  "scales": [
    480,
    512,
    544,
    576,
    608,
    640,
    672,
    704,
    736,
    768,
    800
  ],
  "max_size": 1333,
  "scales2_resize": [
    400,
    500,
    600
  ],
  "scales2_crop": [
    384,
    600
  ]
}
loading annotations into memory...
Done (t=0.50s)
creating index...
index created!


## Compute Acquisition Function

In [ ]:
# ICML Active testing

In [5]:
deep_ensemble_results[0][0]['pred_logits'].shape

(1, 900, 91)

In [9]:
deep_ensemble_results[0][0]['pred_boxes'].shape

(1, 900, 4)

In [7]:
deep_ensemble_results[4][0].keys()

dict_keys(['pred_boxes', 'pred_logits', 'scores', 'labels', 'boxes'])